In [ ]:
import numpy as np
import json
import tensorflow as tf
import os
import sys

import time
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import gym

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=256)])
  except RuntimeError as e:
    print(e)
    
    
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_logical_devices('GPU')

import tf_agents
from tf_agents.environments import py_environment, parallel_py_environment, batched_py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
from tf_agents.networks.q_network import QNetwork
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.metrics import tf_metrics
#from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.drivers.dynamic_episode_driver import DynamicEpisodeDriver
from tf_agents.policies.random_tf_policy import RandomTFPolicy
from tf_agents.utils.common import function, element_wise_squared_loss
from tf_agents.eval.metric_utils import log_metrics
from tf_agents.policies import policy_saver
import logging

import tensorflow.keras as keras

tf.compat.v1.enable_v2_behavior()
import time
import json
import datetime
import copy

from scan_gym import envs
seed=42
tf.random.set_seed(seed)
np.random.seed(seed)

import imp
#import test as nntools
#imp.reload(nntools)

In [ ]:
current_path = os.getcwd()
'''envs_paths = [os.path.join(current_path, 'plants_dataset/scan_plant001/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/scan_plant002/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/scan_plant003/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/scan_plant004/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/scan_plant005/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/scan_plant006/v01/inliers_ratios.npy')
                ]'''

envs_paths = [os.path.join(current_path, 'plants_dataset/real_plant001/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/real_plant001/v03/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/real_plant002/v01/inliers_ratios.npy')]

envs = [suite_gym.load('ScannerEnv-v0',
        gym_kwargs={'setpoint':0.03,'init_pos_inc_rst':True,'inliers_ratios_file':os.path.join(current_path,x)}) for x in envs_paths]

In [ ]:
policy_dir = os.path.join(current_path, 'policy')
policy = tf.compat.v2.saved_model.load(policy_dir)

In [ ]:
def run_episode(tf_env,env,policy):
    state = tf_env.reset()
    last_position = env.current_position
    cont = 0
    time_steps = 3000
    #obs = deque(maxlen=time_steps)

    for i in range(time_steps):
        #obs.append(state.observation[0].numpy())
        cont +=1
        action_step = policy.action(state)
        next_state = tf_env.step(action_step.action)
        #print(cont,':',last_position,state.observation[0].numpy(),   env.action2move(action_step.action.numpy(),state.observation[0][-1].numpy()),state.step_type.numpy()  )#,env.action2move(action_step.action,state.observation[0][-1].numpy()))#action,dir
        state = copy.deepcopy(next_state)
        last_position = env.current_position
        if state.is_last():
            #print('salio',cont)
            break    
    return env.kept_images,env.total_reward,env.num_steps,env.covered_area


def test_model(tf_env,env,policy,match_indexes):
    data = []
    for i in np.arange(0,1440):#range(1440):
        images,t_reward,n_steps,covered_area = run_episode(tf_env,env,policy)
        #######score = get_imageset_error(images,match_indexes,30,covered_area/36)
        #score,stdd= get_error(images,match_indexes,setpoint,covered_area/36)
        score, stdd = get_idx_stats(match_indexes,images)
        print("\rEpisode {} ".format(i),end="")
        sys.stdout.flush()
        #######score = get_image_set_score(images,match_indexes)
        data.append([score,stdd,len(images),t_reward,n_steps,env.total_moves,covered_area])
    return np.array(data)

def test_models(envs,m_idxs,model,scaler=None):
    cdata = []
    cstats = []
    tf_envs = [tf_py_environment.TFPyEnvironment(env) for env in envs]
    
    for i in range(len(envs)):
        env = envs[i]
        tf_env = tf_envs[i]
        match_indexes = np.load(m_idxs[i])
    
        dt = test_model(tf_env,env,policy,match_indexes)
        st = get_stats(dt)
        print(i,':',st)
        cdata.append(dt)
        cstats.append(st)
    return cdata,cstats
    
def get_error(image_set,idx_table,setpoint,coverage_fraction):
    ordered = sorted(image_set)
    errors = []
    for i in range(len(image_set)):
        error = abs(idx_table[ordered[i],ordered[i-1]] - setpoint)
        errors.append(error)
    return (np.mean(errors)*(coverage_fraction)) + (1-coverage_fraction), np.std(errors)

def get_stats(data):
    score = np.mean(data[:,0])
    std = np.mean(data[:,1])
    n_images = np.mean(data[:,2])
    reward = np.mean(data[:,3])
    n_steps = np.mean(data[:,4])
    total_moves = np.mean(data[:,5])
    covered_area = np.mean(data[:,6])
    #how many did not covered he whole area
    n_incomplete = len(data[:,6][ data[:,6] < 8])
    return [score,std,n_images,reward,n_steps,total_moves,covered_area,n_incomplete]

def get_idx_stats(idx_table,image_set):
    ordered = sorted(image_set)
    idxs = []
    for i in range(len(image_set)):
        idx = idx_table[ordered[i],ordered[i-1]]
        idxs.append(idx)
    return np.mean(idxs),np.std(idxs)

In [ ]:
dt,st = test_models(envs,envs_paths,policy)

In [ ]:
#['mean_idx','std','n_images','score','total_steps','n_movements','covered_area','n_incomplete']